In [1]:
import io
import pandas as pd
import re
import numpy as np

with io.open("imdb_urls_new.txt", "r", encoding="utf-8") as my_file:
    movie_file = my_file.read() 

links = []
book_names = []

for line in movie_file.split("\n"):
    arr = line.split('\t<>\t')
    re_obj = re.search("(.*)[?]", 'http://'+arr[1])
    link = re_obj.group(1)
    links.append(link.strip())
    re_obj = re.search("(.*?)(\(|$)", arr[0])
    bname = re_obj.group(1)
    book_names.append(bname.strip())
    
links = np.array(links)
book_names = np.array(book_names)

df = pd.DataFrame(data=links,columns=['Link'])
df['New Book Name'] = pd.DataFrame(data=book_names,columns=['New Book Name'])

dataframe = pd.read_csv('movie_data_new.csv', header=None, names=['Link','Budget','Gross','Opening'])

dataframe['Link'] = dataframe['Link'].map(lambda x: re.search("(.*?)business", x).group(1).strip())

Merged = pd.merge(df, dataframe, how='right', on='Link', suffixes=('_x', '_y'))



In [2]:
Merged = Merged.drop_duplicates()
Merged = Merged.set_index([range(Merged['Link'].values.size)])
book_data = pd.read_csv('goodreads_data_modified.csv', encoding='utf-8')
Merged = pd.merge(Merged, book_data, how='left', on='New Book Name', suffixes=('_x', '_y'))
Merged = Merged.drop('Unnamed: 0', axis=1)
Merged = Merged.drop_duplicates()
Merged = Merged.set_index([range(Merged['Link'].values.size)])


In [3]:
Merged

,Link,New Book Name,Budget,Gross,Opening,Awards List,Book Name,Characters,Genres,Num_reviews,Other editions,Pages,Star votes,Stars,publish_date,Award Count
0,http://www.imdb.com/title/tt0373889/,Harry Potter and the Order of the Phoenix,150000000,292004738,77108414,\nBram Stoker Award for Works for Young Reader...,Harry Potter and the Order of the Phoenix (Har...,"[Sirius Black, Draco Malfoy, Ron Weasley, Petu...","[Childrens, Fantasy, Novels]","25,751",278.0,870 pages,"[1041580, 467679, 172852, 30465, 9227]",4.45,June 21st 2003,8.0
1,http://www.imdb.com/title/tt0373889/,Harry Potter and the Order of the Phoenix,150000000,292004738,77108414,\nBram Stoker Award for Works for Young Reader...,Harry Potter and the Order of the Phoenix (Har...,"[Sirius Black, Draco Malfoy, Ron Weasley, Petu...","[Childrens, Fantasy, Novels]","25,751",278,870 pages,"[1041580, 467679, 172852, 30465, 9227]",4.45,June 21st 2003,8.0
2,http://www.imdb.com/title/tt1099212/,Twilight,37000000,191465414,69637740,"\nGeorgia Peach Book Award (2007), Buxtehuder ...",Twilight (Twilight #1),"[Edward Cullen, Jacob Black, Laurent, Renee, B...","[Young Adult, Fantasy, Romance, Paranormal, Va...","93,258",229,498 pages,"[1258780, 814235, 738275, 407909, 428866]",3.57,October 5th 2005,22.0
3,http://www.imdb.com/title/tt0330373/,Harry Potter and the Goblet of Fire,150000000,290013036,102335066,"\nHugo Award for Best Novel (2001), Mythopoeic...",Harry Potter and the Goblet of Fire (Harry Pot...,"[Sirius Black, Draco Malfoy, Ron Weasley, Petu...","[Fantasy, Young Adult, Fiction, Fantasy, Magic...","27,770",299,734 pages,"[1081875, 463920, 144699, 19224, 6183]",4.51,July 8th 2000,7.0
4,http://www.imdb.com/title/tt0304141/,Harry Potter and the Prisoner of Azkaban,130000000,249358727,93687367,\nBram Stoker Award for Best Work for Young Re...,Harry Potter and the Prisoner of Azkaban (Harr...,"[Sirius Black, Draco Malfoy, Ron Weasley, Petu...","[Fantasy, Young Adult, Fiction]","32,160",331,435 pages,"[1144372, 477102, 158646, 19530, 6204]",4.51,July 8th 1999,12.0
5,http://www.imdb.com/title/tt0241527/,Harry Potter and the Sorcerer's Stone,125000000,317557891,90294621,\nMythopoeic Fantasy Award for Children's Lite...,Harry Potter and the Sorcerer's Stone (Harry P...,"[Draco Malfoy, Ron Weasley, Petunia Dursley, V...","[Fantasy, Young Adult, Fiction]","67,668",156,320 pages,"[2650321, 1054137, 423981, 94447, 67115]",4.43,NaN,24.0
6,http://www.imdb.com/title/tt0295297/,Harry Potter and the Chamber of Secrets,100000000,261970615,88357488,\nMythopoeic Fantasy Award for Children's Lite...,Harry Potter and the Chamber of Secrets (Harry...,"[Draco Malfoy, Ron Weasley, Petunia Dursley, V...","[Fantasy, Young Adult, Fiction, Fantasy, Magic...","29,953",360,341 pages,"[955969, 507584, 230507, 40686, 7721]",4.36,1998,7.0
7,http://www.imdb.com/title/tt0417741/,Harry Potter and the Half-Blood Prince,250000000,301959197,77835727,\nBritish Book of the Year (2006)\n,Harry Potter and the Half-Blood Prince (Harry ...,"[Ron Weasley, Harry Potter, Hermione Granger, ...","[Fantasy, Young Adult, Fiction]","24,934",255,652 pages,"[1059719, 433145, 130525, 20619, 6852]",4.53,July 16th 2005,1.0
8,http://www.imdb.com/title/tt1932888/,Eragon,NaN,NaN,NaN,\nBook Sense Book of the Year Award for Childr...,Eragon (The Inheritance Cycle #1),"[Eragon (The Inheritance Cycle), Saphira (The ...","[Fantasy, Young Adult, Fiction, Fantasy, Drago...","17,413",199,503 pages,"[374199, 313859, 225705, 84229, 48311]",3.84,June 2002,18.0
9,http://www.imdb.com/title/tt0397535/,Memoirs of a Geisha,85000000,57010853,682504,NaN,Memoirs of a Geisha,"[Nitta Sayuri, Mr. Bekku, Mr. Tanaka, Dr. Crab]","[Fiction, Historical Fiction, Romance, Histori...","24,630",201,434 pages,"[522756, 489841, 248098, 57061, 22646]",4.07,September 1997,0.0


In [4]:
dataframe = pd.read_csv('movie_data_releasedate.csv', header=None, names=['Link','Release Date'])

dataframe['Link'] = dataframe['Link'].map(lambda x: re.search("(.*)ref", x).group(1).strip()[:-1])

dataframe = dataframe.drop_duplicates()

In [5]:
dataframe

,Link,Release Date
0,http://www.imdb.com/title/tt0373889/,11 July 2007
1,http://www.imdb.com/title/tt1099212/,21 November 2008
2,http://www.imdb.com/title/tt0330373/,18 November 2005
3,http://www.imdb.com/title/tt0304141/,4 June 2004
4,http://www.imdb.com/title/tt0241527/,16 November 2001
5,http://www.imdb.com/title/tt0295297/,15 November 2002
6,http://www.imdb.com/title/tt0417741/,15 July 2009
7,http://www.imdb.com/title/tt1932888/,NaN
8,http://www.imdb.com/title/tt0397535/,23 December 2005
9,http://www.imdb.com/title/tt0312528/,21 November 2003


In [6]:
Merged = pd.merge(Merged, dataframe, how='left', on='Link', suffixes=('_x', '_y'))

In [7]:
Merged.to_csv('Merged_Dataset_new.csv', encoding='utf-8')

In [8]:
Merged

,Link,New Book Name,Budget,Gross,Opening,Awards List,Book Name,Characters,Genres,Num_reviews,Other editions,Pages,Star votes,Stars,publish_date,Award Count,Release Date
0,http://www.imdb.com/title/tt0373889/,Harry Potter and the Order of the Phoenix,150000000,292004738,77108414,\nBram Stoker Award for Works for Young Reader...,Harry Potter and the Order of the Phoenix (Har...,"[Sirius Black, Draco Malfoy, Ron Weasley, Petu...","[Childrens, Fantasy, Novels]","25,751",278.0,870 pages,"[1041580, 467679, 172852, 30465, 9227]",4.45,June 21st 2003,8.0,11 July 2007
1,http://www.imdb.com/title/tt0373889/,Harry Potter and the Order of the Phoenix,150000000,292004738,77108414,\nBram Stoker Award for Works for Young Reader...,Harry Potter and the Order of the Phoenix (Har...,"[Sirius Black, Draco Malfoy, Ron Weasley, Petu...","[Childrens, Fantasy, Novels]","25,751",278,870 pages,"[1041580, 467679, 172852, 30465, 9227]",4.45,June 21st 2003,8.0,11 July 2007
2,http://www.imdb.com/title/tt1099212/,Twilight,37000000,191465414,69637740,"\nGeorgia Peach Book Award (2007), Buxtehuder ...",Twilight (Twilight #1),"[Edward Cullen, Jacob Black, Laurent, Renee, B...","[Young Adult, Fantasy, Romance, Paranormal, Va...","93,258",229,498 pages,"[1258780, 814235, 738275, 407909, 428866]",3.57,October 5th 2005,22.0,21 November 2008
3,http://www.imdb.com/title/tt0330373/,Harry Potter and the Goblet of Fire,150000000,290013036,102335066,"\nHugo Award for Best Novel (2001), Mythopoeic...",Harry Potter and the Goblet of Fire (Harry Pot...,"[Sirius Black, Draco Malfoy, Ron Weasley, Petu...","[Fantasy, Young Adult, Fiction, Fantasy, Magic...","27,770",299,734 pages,"[1081875, 463920, 144699, 19224, 6183]",4.51,July 8th 2000,7.0,18 November 2005
4,http://www.imdb.com/title/tt0304141/,Harry Potter and the Prisoner of Azkaban,130000000,249358727,93687367,\nBram Stoker Award for Best Work for Young Re...,Harry Potter and the Prisoner of Azkaban (Harr...,"[Sirius Black, Draco Malfoy, Ron Weasley, Petu...","[Fantasy, Young Adult, Fiction]","32,160",331,435 pages,"[1144372, 477102, 158646, 19530, 6204]",4.51,July 8th 1999,12.0,4 June 2004
5,http://www.imdb.com/title/tt0241527/,Harry Potter and the Sorcerer's Stone,125000000,317557891,90294621,\nMythopoeic Fantasy Award for Children's Lite...,Harry Potter and the Sorcerer's Stone (Harry P...,"[Draco Malfoy, Ron Weasley, Petunia Dursley, V...","[Fantasy, Young Adult, Fiction]","67,668",156,320 pages,"[2650321, 1054137, 423981, 94447, 67115]",4.43,NaN,24.0,16 November 2001
6,http://www.imdb.com/title/tt0295297/,Harry Potter and the Chamber of Secrets,100000000,261970615,88357488,\nMythopoeic Fantasy Award for Children's Lite...,Harry Potter and the Chamber of Secrets (Harry...,"[Draco Malfoy, Ron Weasley, Petunia Dursley, V...","[Fantasy, Young Adult, Fiction, Fantasy, Magic...","29,953",360,341 pages,"[955969, 507584, 230507, 40686, 7721]",4.36,1998,7.0,15 November 2002
7,http://www.imdb.com/title/tt0417741/,Harry Potter and the Half-Blood Prince,250000000,301959197,77835727,\nBritish Book of the Year (2006)\n,Harry Potter and the Half-Blood Prince (Harry ...,"[Ron Weasley, Harry Potter, Hermione Granger, ...","[Fantasy, Young Adult, Fiction]","24,934",255,652 pages,"[1059719, 433145, 130525, 20619, 6852]",4.53,July 16th 2005,1.0,15 July 2009
8,http://www.imdb.com/title/tt1932888/,Eragon,NaN,NaN,NaN,\nBook Sense Book of the Year Award for Childr...,Eragon (The Inheritance Cycle #1),"[Eragon (The Inheritance Cycle), Saphira (The ...","[Fantasy, Young Adult, Fiction, Fantasy, Drago...","17,413",199,503 pages,"[374199, 313859, 225705, 84229, 48311]",3.84,June 2002,18.0,NaN
9,http://www.imdb.com/title/tt0397535/,Memoirs of a Geisha,85000000,57010853,682504,NaN,Memoirs of a Geisha,"[Nitta Sayuri, Mr. Bekku, Mr. Tanaka, Dr. Crab]","[Fiction, Historical Fiction, Romance, Histori...","24,630",201,434 pages,"[522756, 489841, 248098, 57061, 22646]",4.07,September 1997,0.0,23 December 2005
